In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
import torch
from torch import nn
%matplotlib inline
import matplotlib.pyplot as plt
from dnaDataloader import expermentDataloader
from dnaDataloader import addData
from dnaModelUtil import MLPModel
from dnaModelUtil import train
from datetime import datetime
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
folder = '/home/khood/GitHub/SNN-DNA-project/Prepocessing/sorted/1800_nM_AR_5000'
oneMinInFPS = 1200
batch_size = 10
num_workers = 0
device

device(type='cuda', index=2)

## MLP running from 1 min to 30 min
get reults under varied movie length 

In [12]:
return_dict = {}
for length in range(1, 31):
    data = expermentDataloader(
        f"{folder}/index.csv",
        f"{folder}", 
        length=oneMinInFPS*length
    )
    rawData = [d for d in data]
    featIn = len(rawData[0][0])
    featIn = len(rawData[0][0])
    trainValidData = []
    testData = []
    addData(testData, trainValidData, rawData, rhsSize=300)


    np.random.shuffle(trainValidData)
    trainData = []
    validData = []
    addData(trainData, validData, trainValidData, rhsSize=int(len(trainValidData)*(1/3)))

    trainDataset = DataLoader(trainData, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True) 
    validDataset = DataLoader(validData, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
    testDataset = DataLoader(testData, batch_size=len(testData), shuffle=True, num_workers=num_workers, pin_memory=True)

    train(trainData=trainDataset, validData=validDataset, name=f"MLP_{length}_min", featIn=featIn, return_dict=return_dict, epochs=25000, device=device, printStatus=True, capacity=1200)

training 10 on cuda:2...
training 20 on cuda:2...5000	           Train Loss:2384.9058 Valid Loss:4724.4316	           Train accuracy:0.7100 Valid accuracy:0.2800           
training 30 on cuda:2...5000	           Train Loss:2183.3748 Valid Loss:4000.0854	           Train accuracy:0.6850 Valid accuracy:0.3400           


In [13]:
return_dict

{'10': {'path': './Models/smallTrain/10.pt', 'acc': 0.29},
 '20': {'path': './Models/smallTrain/20.pt', 'acc': 0.34},
 '30': {'path': './Models/smallTrain/30.pt', 'acc': 0.35}}